In [21]:
# Import dependencies

# Python SQL toolkit and Object Relational Mapper
# engine, inspect, and functions
from sqlalchemy import create_engine, inspect, func

# declarative base because when you want something done right... you don't use automap...
from sqlalchemy.ext.declarative import declarative_base

# columns and data types
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.types import Date, Time

# session for queries
from sqlalchemy.orm import Session

# operations for queries
from sqlalchemy.sql.expression import and_, or_, extract

%matplotlib inline
from matplotlib import style

style.use("fivethirtyeight")
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta

# formatting tool for notebook
%reload_ext lab_black
from pprint import pprint

# Reflect Tables into SQLAlchemy ORM

In [14]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.orm import Session
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, func

In [15]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
conn = engine.connect()
session = Session(bind=engine)
inspector = inspect(engine)

In [16]:
# read table names
table_names = inspector.get_table_names()
table_names

['measurement', 'station']

In [17]:
# show data in tables
for table_name in table_names:
    print(f"TABLE: {table_name}")
    columns = inspector.get_columns(table_name)
    pprint(columns)
    sample = engine.execute(f"SELECT * FROM {table_name} LIMIT 5")
    print(f"DATA FROM TABLE {table_name}: ")
    for row in sample:
        print(row)

TABLE: measurement
[{'autoincrement': 'auto',
  'default': None,
  'name': 'id',
  'nullable': False,
  'primary_key': 1,
  'type': INTEGER()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'station',
  'nullable': True,
  'primary_key': 0,
  'type': TEXT()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'date',
  'nullable': True,
  'primary_key': 0,
  'type': TEXT()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'prcp',
  'nullable': True,
  'primary_key': 0,
  'type': FLOAT()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'tobs',
  'nullable': True,
  'primary_key': 0,
  'type': FLOAT()}]
DATA FROM TABLE measurement: 
(1, 'USC00519397', '2010-01-01', 0.08, 65.0)
(2, 'USC00519397', '2010-01-02', 0.0, 63.0)
(3, 'USC00519397', '2010-01-03', 0.0, 74.0)
(4, 'USC00519397', '2010-01-04', 0.0, 76.0)
(5, 'USC00519397', '2010-01-06', None, 73.0)
TABLE: station
[{'autoincrement': 'auto',
  'default': None,
  'name': 'id',
  'nullable': False,
  'primar

In [18]:
# create classes
Base = declarative_base()


class Measurement(Base):
    __tablename__ = "measurement"
    id = Column(Integer, primary_key=True)
    station = Column(String(30))
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Float)


class Station(Base):
    __tablename__ = "station"
    id = Column(Integer, primary_key=True)
    station = Column(String(30))
    name = Column(String(120))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

# Exploratory Climate Analysis

In [20]:
# find min and max dates in data
min_date, max_date = session.query(
    func.min(Measurement.date), func.max(Measurement.date)
).first()
print(f"Measurement table time range - start date: {min_date} and end date: {max_date}")

Measurement table time range, start date: 2010-01-01 and end date: 2017-08-23


In [26]:
# find dates for the past year
start_date = max_date - relativedelta(years=1)
end_date = max_date
print(f"Dates from the last year - start date: {start_date} and end date {end_date}")

Dates from the last year - start date: 2016-08-23 and end date 2017-08-23
